In [123]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import requests
from finta import TA
# Load environment variables

import datetime as dt
import pytz

_timeNow = dt.datetime.now(pytz.timezone('US/Eastern'))
_1YearsAgo = _timeNow - dt.timedelta(days=360)
_2YearsAgo = _timeNow - dt.timedelta(days=730)
_3YearsAgo = _timeNow - dt.timedelta(days=1095)
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets/v2/assets?asset_class=crypto"
# Create a connection to the API 
api = tradeapi.REST("PKHBLCB7QI73L4912M0G", "Aq2JSZktyKcSKaFWq1VxIPn3EOFJB0ICvB7fdmpE", ALPACA_API_BASE_URL, api_version="v2")
# Set the ticker symbol and the number of shares to buy
ticker = "BTCUSD"
number_of_coins = 1

# Make API call
ETH_prices = api.get_crypto_bars(ticker, '1day',
                         start=_3YearsAgo.isoformat(),
                         end=None,
                         limit=None).df
#Filter data by exchange
ETH_prices = ETH_prices[ETH_prices.exchange == 'CBSE']
# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
# Create a signals_df DataFrame that is a copy of the ixn_df Dataframe
signals_df = ETH_prices.copy()
signals_df.drop(columns=['exchange','volume','trade_count','vwap'],inplace=True)
signals_df.tail()

,close,high,low,open
timestamp,,,,
2022-10-23 05:00:00+00:00,19337.61,19698.10,19070.92,19188.65
2022-10-24 05:00:00+00:00,19341.22,19453.35,19159.35,19337.10
2022-10-25 05:00:00+00:00,20211.45,20420.88,19240.76,19341.22
2022-10-26 05:00:00+00:00,20806.59,21022.81,20136.82,20211.46
2022-10-27 05:00:00+00:00,20247.05,20878.17,20160.72,20806.59


In [124]:
# Set the short window and long windows
fast_window = 15
slow_window = 50

# Add the SMA technical indicators for the short and long windows
signals_df["fast"] = TA.DEMA(signals_df, fast_window)
signals_df["slow"] = TA.DEMA(signals_df, slow_window)

In [125]:

signals_df[["fast","slow"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [126]:
signals_df.head()

,close,high,low,open,fast,slow
timestamp,,,,,,
2019-10-29 05:00:00+00:00,9257.38,9431.36,9223.27,9431.36,9257.380000,9257.380000
2019-10-30 05:00:00+00:00,9088.70,9213.00,9058.62,9164.45,9125.434756,9129.200068
2019-10-31 05:00:00+00:00,9098.49,9160.40,9057.27,9159.96,9099.013196,9102.046585
2019-11-01 05:00:00+00:00,9253.12,9299.99,9071.02,9098.49,9169.134440,9162.967724
2019-11-02 05:00:00+00:00,9283.66,9380.00,9251.52,9308.52,9219.236944,9207.584807


In [127]:
# Set the Signal column
signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the fast window is greater than (or crosses over) the slow Window
# where 0 is when the fast window is under the slow window
signals_df["Signal"][fast_window:] = np.where(
    signals_df["fast"][fast_window:] > signals_df["slow"][fast_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df.iloc[95:105, :]

,close,high,low,open,fast,slow,Signal,Entry/Exit
timestamp,,,,,,,,
2020-02-01 06:00:00+00:00,9300.14,9380.18,9140.00,9380.18,9293.554655,8731.514157,1.0,0.0
2020-02-02 06:00:00+00:00,9323.50,9480.00,9277.77,9300.14,9342.896353,8794.485070,1.0,0.0
2020-02-03 06:00:00+00:00,9280.49,9398.84,9220.00,9392.62,9370.289998,8849.942118,1.0,0.0
2020-02-04 06:00:00+00:00,9246.02,9260.00,9150.00,9164.33,9381.781190,8898.998681,1.0,0.0
2020-02-05 06:00:00+00:00,9633.27,9650.00,9521.47,9613.91,9479.219705,8975.053078,1.0,0.0
2020-02-06 06:00:00+00:00,9742.90,9840.66,9728.00,9763.01,9581.922629,9054.729964,1.0,0.0
2020-02-07 06:00:00+00:00,9808.05,9885.00,9713.99,9742.90,9678.339339,9134.312359,1.0,0.0
2020-02-08 06:00:00+00:00,10111.00,10145.33,9888.76,9905.64,9825.189057,9232.367097,1.0,0.0
2020-02-09 06:00:00+00:00,10168.35,10178.91,10017.00,10111.01,9955.528047,9328.576808,1.0,0.0


In [128]:

signals_df["Signal"].value_counts()

signals_df["Entry/Exit"].value_counts()

 0.0    1061
 1.0      17
-1.0      16
Name: Entry/Exit, dtype: int64

In [129]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[["fast", "slow"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(
    title="BTC Algorithm - Double exponential moving average over price",
    yformatter='%.0f'
)

:Overlay
   .Curve.I          :Curve   [timestamp]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [130]:
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [131]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['fast', 'slow']].shift().dropna()

# Review the DataFrame
X.head()

,fast,slow
timestamp,,
2019-10-30 05:00:00+00:00,9257.380000,9257.380000
2019-10-31 05:00:00+00:00,9125.434756,9129.200068
2019-11-01 05:00:00+00:00,9099.013196,9102.046585
2019-11-02 05:00:00+00:00,9169.134440,9162.967724
2019-11-03 05:00:00+00:00,9219.236944,9207.584807


In [132]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

1.0    587
0.0    508
Name: Signal, dtype: int64

In [133]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2019-10-30 05:00:00+00:00


In [134]:
# Select the ending period for the training data with an offset of 24 months
training_end = X.index.min() + DateOffset(months=24)

# Display the training end date
print(training_end)

2021-10-30 05:00:00+00:00


In [135]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
display(X_train.head())
display(X_train.tail())

,fast,slow
timestamp,,
2019-10-30 05:00:00+00:00,9257.380000,9257.380000
2019-10-31 05:00:00+00:00,9125.434756,9129.200068
2019-11-01 05:00:00+00:00,9099.013196,9102.046585
2019-11-02 05:00:00+00:00,9169.134440,9162.967724
2019-11-03 05:00:00+00:00,9219.236944,9207.584807


,fast,slow
timestamp,,
2021-10-26 05:00:00+00:00,64005.199964,59905.708259
2021-10-27 05:00:00+00:00,63738.712953,60238.633417
2021-10-28 05:00:00+00:00,62945.462226,60367.251213
2021-10-29 05:00:00+00:00,62906.536167,60691.207247
2021-10-30 05:00:00+00:00,62843.450649,60987.694048


In [136]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())

,fast,slow
timestamp,,
2021-10-31 05:00:00+00:00,62764.932196,61259.229100
2021-11-01 05:00:00+00:00,62475.875065,61441.849733
2021-11-02 05:00:00+00:00,62441.704370,61680.325219
2021-11-03 05:00:00+00:00,62709.110332,62001.410087
2021-11-04 05:00:00+00:00,62896.014423,62292.741132


,fast,slow
timestamp,,
2022-10-23 05:00:00+00:00,19159.099339,18776.488953
2022-10-24 05:00:00+00:00,19186.917962,18780.143557
2022-10-25 05:00:00+00:00,19211.417252,18784.652042
2022-10-26 05:00:00+00:00,19436.162845,18856.584816
2022-10-27 05:00:00+00:00,19759.833489,18970.832433


In [137]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [138]:
from sklearn import svm
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()

# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:10]
        

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [139]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

         0.0       0.90      0.67      0.77       181
         1.0       0.74      0.93      0.82       181

    accuracy                           0.80       362
   macro avg       0.82      0.80      0.80       362
weighted avg       0.82      0.80      0.80       362



In [140]:
# Use the pct_change function to generate returns from close prices
signals_df["Actual Returns"] = signals_df["close"].pct_change()

# Create an empty predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
predictions_df['SVM Returns'] = signals_df["Actual Returns"]*predictions_df["Predicted"]

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,Predicted,Actual Returns,SVM Returns
timestamp,,,
2021-10-31 05:00:00+00:00,0.0,-0.014460,-0.0
2021-11-01 05:00:00+00:00,0.0,0.015307,0.0
2021-11-02 05:00:00+00:00,0.0,0.021421,0.0
2021-11-03 05:00:00+00:00,0.0,-0.001301,-0.0
2021-11-04 05:00:00+00:00,0.0,-0.028369,-0.0


,Predicted,Actual Returns,SVM Returns
timestamp,,,
2022-10-23 05:00:00+00:00,1.0,0.007669,0.007669
2022-10-24 05:00:00+00:00,1.0,0.000187,0.000187
2022-10-25 05:00:00+00:00,1.0,0.044994,0.044994
2022-10-26 05:00:00+00:00,1.0,0.029446,0.029446
2022-10-27 05:00:00+00:00,1.0,-0.026892,-0.026892


In [141]:
# Plot the actual returns versus the SVM strategy returns 
(1 + predictions_df[["Actual Returns", "SVM Returns"]]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [142]:
# Import a new classifier from SKLearn
from sklearn import tree

# Initiate the model instance
dtc_model=tree.DecisionTreeClassifier()

In [143]:
# Fit the model using the training data
dtc_model = dtc_model.fit(X_train_scaled,y_train)

# Use the testing dataset to generate the predictions for the new model
dtc_pred = dtc_model.predict(X_test_scaled)
# Review the model's predicted values
dtc_pred[:10]

array([1., 1., 0., 1., 1., 1., 0., 1., 1., 1.])

In [144]:
# Use a classification report to evaluate the model using the predictions and testing data
dtc_testing_report = classification_report(y_test, dtc_pred)

# Print the classification report
print(dtc_testing_report)

              precision    recall  f1-score   support

         0.0       0.91      0.46      0.61       181
         1.0       0.64      0.96      0.77       181

    accuracy                           0.71       362
   macro avg       0.78      0.71      0.69       362
weighted avg       0.78      0.71      0.69       362



In [145]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
final_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
final_df["Predicted"]=dtc_pred

# Add the actual returns to the DataFrame
final_df["Actual Returns"]=signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
final_df["DTC Returns"]=final_df["Actual Returns"]*final_df["Predicted"]

# Review the DataFrame
display(final_df.head())
display(final_df.tail())

,Predicted,Actual Returns,DTC Returns
timestamp,,,
2021-10-31 05:00:00+00:00,1.0,-0.014460,-0.014460
2021-11-01 05:00:00+00:00,1.0,0.015307,0.015307
2021-11-02 05:00:00+00:00,0.0,0.021421,0.000000
2021-11-03 05:00:00+00:00,1.0,-0.001301,-0.001301
2021-11-04 05:00:00+00:00,1.0,-0.028369,-0.028369


,Predicted,Actual Returns,DTC Returns
timestamp,,,
2022-10-23 05:00:00+00:00,1.0,0.007669,0.007669
2022-10-24 05:00:00+00:00,1.0,0.000187,0.000187
2022-10-25 05:00:00+00:00,1.0,0.044994,0.044994
2022-10-26 05:00:00+00:00,1.0,0.029446,0.029446
2022-10-27 05:00:00+00:00,1.0,-0.026892,-0.026892


In [146]:
# Plot the actual returns versus the strategy returns
(1 + final_df[["Actual Returns", "DTC Returns"]]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)